In [27]:
import pandas as pd
import numpy as np

# FILEPATH: /home/roland/Schreibtisch/Capstone/comparison/comp.ipynb
files = ['AngleMap', 'xi_yi_values','Res', 'Raw_ROIsMask', 'Raw_Tophat', 'Raw_Mask', 'Raw_BGSub']
python_files = []
matlab_files = []

for filename in files:
    filepath = f"{filename}.csv"
    filepath_mat = f"{filename}_mat.csv"
    tmp = pd.read_csv(filepath)
    tmp_mat = pd.read_csv(filepath_mat)
    python_files.append(tmp)
    matlab_files.append(tmp_mat)

# quantify differences between python and matlab files
differences = []
max_values = []
for python_df, matlab_df in zip(python_files, matlab_files):
    # Flatten the DataFrames into 1D arrays
    python_array = python_df.values.flatten()
    matlab_array = matlab_df.values.flatten()

    # Calculate the mean absolute difference
    mean_abs_diff = np.mean(np.abs(python_array - matlab_array))
    differences.append(mean_abs_diff)

    # Calculate the max value
    max_value = max(python_df.values.max(), matlab_df.values.max())
    max_values.append(max_value)

# Print the differences and max values
for filename, mean_abs_diff, max_value in zip(files, differences, max_values):
    print(f"For {filename}:")
    print(f"Mean absolute difference: {mean_abs_diff}")
    print(f"Max value: {max_value}\n")

For AngleMap:
Mean absolute difference: 0.1534090909090909
Max value: 180.0

For xi_yi_values:
Mean absolute difference: 11.49074733836404
Max value: 869.77638454924

For Res:
Mean absolute difference: 2.4894704949224484e-07
Max value: 114.559493091515

For Raw_ROIsMask:
Mean absolute difference: 0.000946969696969697
Max value: 1.0

For Raw_Tophat:
Mean absolute difference: 11.681450500965163
Max value: 3445

For Raw_Mask:
Mean absolute difference: 0.0017250360020835249
Max value: 1.0

For Raw_BGSub:
Mean absolute difference: 4.1303520544167664
Max value: 3981



In [28]:
difference = matlab_files[0] - python_files[0]
difference
print(matlab_files[0].isna().any().any())
print(python_files[0].isna().any().any())
print(matlab_files[0].columns.equals(python_files[0].columns))
print(matlab_files[0].index.equals(python_files[0].index))
python_files[0].shape


False
False
False
True


(32, 33)

In [29]:
import os
# do ls to get all files in directory
# Get the current directory
directory = os.getcwd()

# List all files in the directory
files = os.listdir(directory)

# Print the list of files
for file in files:
    print(file)



Raw_BGSub.csv
Raw_ROIsMask_mat.csv
AngleMap_mat.csv
Res.csv
Raw_Mask.csv
raw.csv
AngleMap.csv
subplot.pdf
sd_plot.png
Raw_Mask_mat.csv
rot.csv
Raw_Tophat_mat.csv
Res_mat.csv
xi_yi_values.csv
comp.ipynb
xi_yi_values_mat.csv
subplot_mat.pdf
rot_after.csv
Raw_ROIsMask.csv
res.csv
Raw_BGSub_mat.csv
Raw_Tophat.csv
data.ods
